<a href="https://colab.research.google.com/github/isaacbull/Data-Science-zikspot/blob/main/visualizing_panel_data_3_dimensions%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction
This Jupyter Notebook explores various methods for creating and visualizing multi-dimensional data in Python. It demonstrates the use of libraries like Pandas, Panel, Xarray, and Bokeh to handle and display data with more than two dimensions. The notebook covers the following topics:

- Creating Multi-Dimensional Data: We will use Pandas and Xarray to generate sample data representing stock prices over time, with dimensions like quarters, metrics (open/close), stock symbols, and potentially geographical locations (latitude/longitude) or other relevant factors.

- Data Exploration with Panel: Panel is used to build interactive dashboards for exploring our multi-dimensional datasets. We'll create widgets to select different aspects of the data (e.g., quarter, metric) and dynamically update the displayed information.

- Visualization with Bokeh: Bokeh provides powerful tools to create interactive plots. We'll use it to generate a simple line and scatter plot, demonstrating how to link data from a ColumnDataSource to visual elements.

- Hvplot for Interactive Plotting: Hvplot, built on top of Bokeh and HoloViews, simplifies the creation of interactive visualizations from Pandas DataFrames and Xarray datasets. We will use it to quickly generate an interactive line plot.

In [10]:
# prompt: install pandas panel and panel4d module

!pip install -q panel


In [7]:
# prompt: import panel and use it to create data

import pandas as pd
import numpy as np
import panel as pn

# Sample data
dates = pd.date_range('20210101', periods=3)
data = {
    'Q1': {
        'Open': pd.DataFrame(np.random.rand(3, 2), index=dates, columns=['AAPL', 'MSFT']),
        'Close': pd.DataFrame(np.random.rand(3, 2), index=dates, columns=['AAPL', 'MSFT']),
    },
    'Q2': {
        'Open': pd.DataFrame(np.random.rand(3, 2), index=dates, columns=['AAPL', 'MSFT']),
        'Close': pd.DataFrame(np.random.rand(3, 2), index=dates, columns=['AAPL', 'MSFT']),
    }
}

# Create a Panel4D (Note: Panel4D is deprecated in recent pandas versions)
# Use a dictionary of DataFrames instead
panel_data = data


# Example using Panel to display one of the DataFrames
pn.extension()

# Select a specific DataFrame (e.g., Q1 Open)
df_to_display = panel_data['Q1']['Open']


# Display the DataFrame using Panel's DataFrame component
pn.widgets.DataFrame(df_to_display, name="Stock Prices")

<ipython-input-7-8d59aef03c06>:26: UserWarning: Using Panel interactively in Colab notebooks requires the jupyter_bokeh package to be installed. Install it with:

    !pip install jupyter_bokeh

and try again.
  pn.extension()


DataFrame(name='Stock Prices', value=              ...)

In [13]:


pn.extension()

# Iterate through the panel_data and display each DataFrame
for quarter, quarter_data in panel_data.items():
  for metric, df in quarter_data.items():
    print(f"Displaying data for {quarter} - {metric}:")
    print(df)
    pn.widgets.DataFrame(df, name=f"{quarter} - {metric}")

<ipython-input-13-ff82b843b67c>:1: UserWarning: Using Panel interactively in Colab notebooks requires the jupyter_bokeh package to be installed. Install it with:

    !pip install jupyter_bokeh

and try again.
  pn.extension()


Displaying data for Q1 - Open:
                AAPL      MSFT
2021-01-01  0.296644  0.147741
2021-01-02  0.269018  0.461961
2021-01-03  0.729377  0.421906
Displaying data for Q1 - Close:
                AAPL      MSFT
2021-01-01  0.361189  0.799098
2021-01-02  0.549646  0.883232
2021-01-03  0.016723  0.061860
Displaying data for Q2 - Open:
                AAPL      MSFT
2021-01-01  0.324993  0.675936
2021-01-02  0.761503  0.325197
2021-01-03  0.731113  0.428311
Displaying data for Q2 - Close:
                AAPL      MSFT
2021-01-01  0.761909  0.835053
2021-01-02  0.277933  0.450767
2021-01-03  0.413220  0.125363


In [8]:
# prompt: import panel4d and use it to create data

import pandas as pd
import numpy as np
import panel as pn

pn.extension()

# Sample data (using a dictionary of DataFrames)
dates = pd.date_range('20210101', periods=3)
panel_data = {
    'Q1': {
        'Open': pd.DataFrame(np.random.rand(3, 2), index=dates, columns=['AAPL', 'MSFT']),
        'Close': pd.DataFrame(np.random.rand(3, 2), index=dates, columns=['AAPL', 'MSFT']),
    },
    'Q2': {
        'Open': pd.DataFrame(np.random.rand(3, 2), index=dates, columns=['AAPL', 'MSFT']),
        'Close': pd.DataFrame(np.random.rand(3, 2), index=dates, columns=['AAPL', 'MSFT']),
    }
}


# Interactive Panel elements for exploration
quarter_selector = pn.widgets.Select(name='Quarter', options=list(panel_data.keys()))
price_type_selector = pn.widgets.Select(name='Price Type', options=['Open', 'Close'])


def update_data(event):
    quarter = quarter_selector.value
    price_type = price_type_selector.value
    df = panel_data[quarter][price_type]
    data_table.value = df


quarter_selector.param.watch(update_data, 'value')
price_type_selector.param.watch(update_data, 'value')


# Initial DataFrame display
df = panel_data['Q1']['Open']
data_table = pn.widgets.DataFrame(df, name="Stock Prices")

# Layout using Panel
pn.Column(
    quarter_selector,
    price_type_selector,
    data_table
).servable()

<ipython-input-8-f3f421c48e86>:7: UserWarning: Using Panel interactively in Colab notebooks requires the jupyter_bokeh package to be installed. Install it with:

    !pip install jupyter_bokeh

and try again.
  pn.extension()


Column
    [0] Select(name='Quarter', options=['Q1', 'Q2'], value='Q1')
    [1] Select(name='Price Type', options=['Open', 'Close'], value='Open')
    [2] DataFrame(name='Stock Prices', value=              ...)

In [21]:
# prompt: create dataframes of 4 dimensions using pandas multiindex method

import pandas as pd
import numpy as np

# Sample data (replace with your actual data)
arrays = [
    np.array(['Q1', 'Q1', 'Q2', 'Q2']),
    np.array(['Open', 'Close', 'Open', 'Close'])
]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=['Quarter', 'Metric'])

columns = pd.MultiIndex.from_product([['AAPL', 'MSFT'], ['Price', 'Volume']], names=['Stock', 'Data'])
data = np.random.rand(4, 4)
df = pd.DataFrame(data, index=index, columns=columns)

df

Stock               AAPL                MSFT          
Data               Price    Volume     Price    Volume
Quarter Metric                                        
Q1      Open    0.602180  0.100787  0.735051  0.944426
        Close   0.031862  0.916205  0.711661  0.988541
Q2      Open    0.087971  0.550474  0.011943  0.972696
        Close   0.425815  0.961014  0.651924  0.388902

In [23]:
# prompt: create dataframes of 4 dimensions using xarray

import xarray as xr
import pandas as pd
import numpy as np

# Sample data (replace with your actual data)
time = pd.date_range("2023-01-01", periods=3)
lat = [30, 40, 50]
lon = [-100, -90, -80]
var = np.random.rand(3, 3, 3)


# Create xarray DataArray
da = xr.DataArray(
    var,
    coords=[time, lat, lon],
    dims=['time', 'lat', 'lon'],
    name="my_variable",
)

# Add a fourth dimension (e.g., level)
levels = [1000, 500, 200]  # Example levels
da = da.expand_dims({"level": levels})

print(f"printing da: {da}\n")

# Access data
print(f"\nprinting da.sel:\n")
print(da.sel(time="2023-01-02", level=500))

# Other operations

print(f"\nprinting da.mean:\n")
print(da.mean(dim="time"))


printing da: <xarray.DataArray 'my_variable' (level: 3, time: 3, lat: 3, lon: 3)> Size: 648B
array([[[[0.3123438 , 0.34157667, 0.12391926],
         [0.16990703, 0.04451195, 0.31507417],
         [0.56386582, 0.75222579, 0.52990453]],

        [[0.23316721, 0.54155171, 0.39402327],
         [0.08653376, 0.11040318, 0.13320994],
         [0.51130163, 0.29453904, 0.36318639]],

        [[0.97165132, 0.5788966 , 0.87602855],
         [0.84117268, 0.51587454, 0.23349481],
         [0.70662808, 0.01261938, 0.24639141]]],


       [[[0.3123438 , 0.34157667, 0.12391926],
         [0.16990703, 0.04451195, 0.31507417],
         [0.56386582, 0.75222579, 0.52990453]],

        [[0.23316721, 0.54155171, 0.39402327],
         [0.08653376, 0.11040318, 0.13320994],
         [0.51130163, 0.29453904, 0.36318639]],

        [[0.97165132, 0.5788966 , 0.87602855],
         [0.84117268, 0.51587454, 0.23349481],
         [0.70662808, 0.01261938, 0.24639141]]],


       [[[0.3123438 , 0.34157667, 0.12391926]

In [28]:
# prompt: create a simple panel dashboard

import pandas as pd
import numpy as np
import panel as pn
import xarray as xr

pn.extension()

# Sample data (replace with your actual data)
time = pd.date_range("2023-01-01", periods=3)
lat = [30, 40, 50]
lon = [-100, -90, -80]
var = np.random.rand(3, 3, 3)

# Create xarray DataArray
da = xr.DataArray(
    var,
    coords=[time, lat, lon],
    dims=['time', 'lat', 'lon'],
    name="my_variable",
)

# Add a fourth dimension (e.g., level)
levels = [1000, 500, 200]  # Example levels
da = da.expand_dims({"level": levels})

# Interactive Panel elements for exploration
time_selector = pn.widgets.DatePicker(name='Time', value=da.time[0].values)
level_selector = pn.widgets.Select(name='Level', options=levels, value=levels[0])

def update_data(event):
    time_val = pd.Timestamp(time_selector.value)
    level_val = level_selector.value
    selected_data = da.sel(time=time_val, level=level_val)
    data_table.value = selected_data.to_dataframe()


time_selector.param.watch(update_data, 'value')
level_selector.param.watch(update_data, 'value')


# Initial DataFrame display
initial_data = da.sel(time=da.time[0], level=levels[0])
data_table = pn.widgets.DataFrame(initial_data.to_dataframe(), name="Data")

# Layout using Panel
pn.Column(
    time_selector,
    level_selector,
    data_table
).servable()

<ipython-input-28-4fe8b8a1caf7>:8: UserWarning: Using Panel interactively in Colab notebooks requires the jupyter_bokeh package to be installed. Install it with:

    !pip install jupyter_bokeh

and try again.
  pn.extension()


Column
    [0] DatePicker(name='Time', value=datetime.date(2023, 1, 1))
    [1] Select(name='Level', options=[1000, 500, 200], value=1000)
    [2] DataFrame(name='Data', value=          level  ...)

In [31]:
# prompt: install hvplot and all its dependencies

!pip install -q hvplot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 3.4 MB/s eta 0:00:00


In [32]:
import hvplot.pandas
import pandas as pd

# Sample data
data = pd.DataFrame({'x': range(10), 'y': [i**2 for i in range(10)]})

# Create an interactive plot
interactive_plot = data.hvplot(x='x', y='y', kind='line')
interactive_plot


:Curve   [x]   (y)

In [35]:
# prompt: install jupyter bokeh

!pip install -q bokeh


In [38]:
# prompt: use bokeh to create

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

# Sample data (replace with your actual data)
x = [1, 2, 3, 4, 5]
y = [6, 7, 2, 4, 5]

# Create a ColumnDataSource
source = ColumnDataSource(data=dict(x=x, y=y))

# Create a figure
p = figure(
    title="Simple Bokeh Plot",
    x_axis_label="X-axis",
    y_axis_label="Y-axis",
    width=400,  # Set width
    height=400  # Set height
)

# Add a line renderer
p.line(x='x', y='y', source=source, line_width=2, color="blue")

# Add circle markers
p.circle(x='x', y='y', source=source, size=10, color="red", fill_alpha=0.5)

# Show the plot
show(p)